In [1]:
import pandas as pd
from theano.sandbox import cuda
import utils; reload(utils)
from utils import *
from __future__ import division, print_function


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using Theano backend.


In [2]:
import numpy as np

In [3]:
df_train = pd.read_csv('train.csv')
df_train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [4]:
df_test = pd.read_csv('test.csv')
df_test

,id,comment_text
0,6044863,==Orphaned non-free media (Image:41cD1jboEvL. ...
1,6102620,::Kentuckiana is colloquial. Even though the ...
2,14563293,"Hello fellow Wikipedians,\nI have just modifie..."
3,21086297,"AKC Suspensions \nThe Morning Call - Feb 24, 2..."
4,22982444,== [WIKI_LINK: Talk:Celts] ==
5,24388733,Hello Xdes! [WIKI_LINK: Wikipedia:Introduction...
6,26195914,"Hi there, I'm . I just wanted to let you know ..."
7,31769073,::I don't think a stuffed arm really counts as...
8,35289443,==Was it ever really a single in the UK?==
9,38393350,Thanks for uploading [WIKI_LINK: Image:Cloudco...


In [4]:
df_x = df_train['comment_text']
df_y = df_train.drop('comment_text', axis=1)

In [5]:
df_y = df_y.drop('id', axis=1)
df_y

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,1,1,1,0,1,0
7,0,0,0,0,0,0
8,0,0,0,0,0,0
9,0,0,0,0,0,0


In [6]:
X_train = df_x.as_matrix()
y_train = df_y.as_matrix()

In [7]:
X_train.shape

(159571,)

In [8]:
X_train

array([ "Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
       "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",
       ..., 'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.',
       'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.',
       '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of community goes

In [9]:
vocab_list = []
for i in range(len(X_train)):
    if i%5000 == 0:
        print(i)
    temp_list = X_train[i].split(' ')
    for j in range(len(temp_list)):
        temp_list[j] = temp_list[j].lower()
        if temp_list[j] not in vocab_list:
            vocab_list.append(temp_list[j])

vocab_list

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000


['explanation\nwhy',
 'the',
 'edits',
 'made',
 'under',
 'my',
 'username',
 'hardcore',
 'metallica',
 'fan',
 'were',
 'reverted?',
 'they',
 "weren't",
 'vandalisms,',
 'just',
 'closure',
 'on',
 'some',
 'gas',
 'after',
 'i',
 'voted',
 'at',
 'new',
 'york',
 'dolls',
 'fac.',
 'and',
 'please',
 "don't",
 'remove',
 'template',
 'from',
 'talk',
 'page',
 'since',
 "i'm",
 'retired',
 'now.89.205.38.27',
 "d'aww!",
 'he',
 'matches',
 'this',
 'background',
 'colour',
 'seemingly',
 'stuck',
 'with.',
 'thanks.',
 '',
 '(talk)',
 '21:51,',
 'january',
 '11,',
 '2016',
 '(utc)',
 'hey',
 'man,',
 'really',
 'not',
 'trying',
 'to',
 'edit',
 'war.',
 "it's",
 'that',
 'guy',
 'is',
 'constantly',
 'removing',
 'relevant',
 'information',
 'talking',
 'me',
 'through',
 'instead',
 'of',
 'page.',
 'seems',
 'care',
 'more',
 'about',
 'formatting',
 'than',
 'actual',
 'info.',
 '"\nmore\ni',
 "can't",
 'make',
 'any',
 'real',
 'suggestions',
 'improvement',
 '-',
 'wondered'

In [10]:
random = X_train[1].split(' ')
random

["D'aww!",
 'He',
 'matches',
 'this',
 'background',
 'colour',
 "I'm",
 'seemingly',
 'stuck',
 'with.',
 'Thanks.',
 '',
 '(talk)',
 '21:51,',
 'January',
 '11,',
 '2016',
 '(UTC)']

In [11]:
vocab_list[:10]

['explanation\nwhy',
 'the',
 'edits',
 'made',
 'under',
 'my',
 'username',
 'hardcore',
 'metallica',
 'fan']

In [12]:
#shuffling vocab_list
from random import shuffle
vocabulary = shuffle(vocab_list)


In [13]:
vocab_list[:10]

['juryman',
 "clark's",
 'ptolemy,',
 '2112',
 'seniority;',
 '\nstorm,',
 '\n\nawesome',
 'request"".',
 'child-brides(woody',
 'options;']

In [14]:
import pickle
with open('vocab_list.txt', "wb") as fp:
    pickle.dump(vocab_list, fp)
    
with open("vocab_list.txt", "rb") as fp:
    b = pickle.load(fp)

b

['juryman',
 "clark's",
 'ptolemy,',
 '2112',
 'seniority;',
 '\nstorm,',
 '\n\nawesome',
 'request"".',
 'child-brides(woody',
 'options;',
 'whommoney',
 'longhorns!',
 'dvansickle"',
 'thoughts:\nyes,',
 'delicious!""',
 'reach""',
 'category:interwar',
 'questions.\n\nif',
 'liners',
 'gurkani,',
 'spammer(or',
 "'chopper",
 'roe\xe2\x80\x99s',
 'me.\nright',
 'sidewalls',
 'speedweek,',
 'athame,',
 '""baghdadi""',
 "'mock'",
 'tamaz',
 'open\xe2\x80\xa6',
 '\n\naftermath',
 'blessed.\n\nsho-ho',
 'wednesday',
 'advise?',
 'head!!!',
 'qualiies',
 'fraser',
 'garg',
 'blavatsky.',
 'orsini',
 'degraded.',
 'image:3059240676a3061064212b640781442m.jpg\n\nthanks',
 '(good-cop/bad-cop)',
 'riverlife.',
 'wikipedia:images_and_media_for_deletion',
 'whittington',
 'fortifications,',
 'm\xc3\xael\xc3\xael\xc3\xae',
 'fighting!',
 'michael-mike1',
 'occupation-republic',
 'fear-mongering.',
 'fact.\nhttp://www.world-action.co.uk/independent.html',
 '(q.v.)',
 "university'.",
 'kuma',
 'su

In [17]:
test_list = []
temp_list = []
for i in range(len(vocab_list)):
    temp_list = vocab_list[i].split('\n')
    for j in range (len(temp_list)):
        test_list.append(temp_list[j])

test_list

['juryman',
 "clark's",
 'ptolemy,',
 '2112',
 'seniority;',
 '',
 'storm,',
 '',
 '',
 'awesome',
 'request"".',
 'child-brides(woody',
 'options;',
 'whommoney',
 'longhorns!',
 'dvansickle"',
 'thoughts:',
 'yes,',
 'delicious!""',
 'reach""',
 'category:interwar',
 'questions.',
 '',
 'if',
 'liners',
 'gurkani,',
 'spammer(or',
 "'chopper",
 'roe\xe2\x80\x99s',
 'me.',
 'right',
 'sidewalls',
 'speedweek,',
 'athame,',
 '""baghdadi""',
 "'mock'",
 'tamaz',
 'open\xe2\x80\xa6',
 '',
 '',
 'aftermath',
 'blessed.',
 '',
 'sho-ho',
 'wednesday',
 'advise?',
 'head!!!',
 'qualiies',
 'fraser',
 'garg',
 'blavatsky.',
 'orsini',
 'degraded.',
 'image:3059240676a3061064212b640781442m.jpg',
 '',
 'thanks',
 '(good-cop/bad-cop)',
 'riverlife.',
 'wikipedia:images_and_media_for_deletion',
 'whittington',
 'fortifications,',
 'm\xc3\xael\xc3\xael\xc3\xae',
 'fighting!',
 'michael-mike1',
 'occupation-republic',
 'fear-mongering.',
 'fact.',
 'http://www.world-action.co.uk/independent.html',

In [18]:
saved_vocab_list = test_list
with open('final_list.txt', "wb") as fp:
    pickle.dump(saved_vocab_list, fp)

In [36]:
X_train[0] = 
test = test.lower()
test_split = test.split(' ')
'''
for i in range(len(test_split)):
    word = ' ' + test_split[i]
    test = test.replace(word, str(vocab_list.index(test_split[i])))
    print('number: ', str(vocab_list.index(test_split[i])))
    print('word: ', test_split[i])
'''

test_split

['456614',
 '324675',
 '317793',
 '360740',
 '451048',
 '242308',
 '534125',
 '76021',
 '537126',
 '105138',
 '209993',
 '193431',
 '324675y',
 "209993n't",
 '258967',
 '443204',
 '188260',
 '297329',
 '275992',
 '234716',
 '154279',
 '26568',
 '278614',
 '369918',
 '252391',
 '112143',
 '484122',
 '430413',
 '26720',
 '363374',
 "d297329't",
 '487117',
 '324675',
 'templ369918e',
 '21865',
 '324675',
 '423299',
 '233786',
 's26568nce',
 "26568'm",
 'ret26568red',
 '22259']

547300

In [41]:
test = test.replace('Nonsense', 'text')
test

"text?  kiss off, geek. what I said is true.  I'll have your account terminated."